In [ ]:
from choice_learn.datasets import load_modecanada

transport_df = load_modecanada(as_frame=True)
print(f"Dataset shape: {transport_df.shape}")
display(transport_df.head(8))

In [ ]:
# Question 1a
case1 = transport_df[transport_df["case"] == 1]
print("\nCase 1:")
print(case1[["case", "alt", "choice"]])
print(f"Number of alternatives available: {len(case1)}")
print(f"Chosen alternative: {case1[case1['choice'] == 1]['alt'].values[0]}")


In [ ]:
# Question 1b

from choice_learn.data import ChoiceDataset

canada_dataset = ChoiceDataset.from_single_long_df(
    df=transport_df,
    items_id_column="alt",
    choices_id_column="case",
    choices_column="choice",
    shared_features_columns=["income"],
    items_features_columns=["cost", "freq", "ovt", "ivt"],
    choice_format="one_zero"
)
print(canada_dataset.summary())

Question 1c
In-vehicle time (ivt) has ALTERNATIVE-SPECIFIC coefficients because the subjective experience of time differs. Obviously, on a plane people can sleep or watch movies but in a car you must focus on driving and suffer from the tiredness. Moreover, the flexibility level on a train is more than a car but less than a plane. 
Therefore, a single shared coefficient would force the same disutility across different mode which does not make any sense.

In [ ]:
# Question 1d

from choice_learn.models import ConditionalLogit

model = ConditionalLogit()

# Shared coefficients — one β for all modes
# The assumption is that the effect of cost (or freq, ovt) on utility
# doesn't depend on which mode you're looking at
model.add_shared_coefficient(coefficient_name="beta_cost", feature_name="cost", items_indexes=[0, 1, 2, 3])
model.add_shared_coefficient(coefficient_name="beta_freq", feature_name="freq", items_indexes=[0, 1, 2, 3])
model.add_shared_coefficient(coefficient_name="beta_ovt",  feature_name="ovt",  items_indexes=[0, 1, 2, 3])

# Alternative-specific ivt — each mode has its own coefficient
# because the subjective experience of time varies by mode
model.add_coefficients(coefficient_name="beta_ivt",    feature_name="ivt",       items_indexes=[0, 1, 2, 3])

# Alternative-specific intercepts and income effects
# Car (index 2) is the reference category, so it's excluded here
# All other coefficients are interpreted relative to car
model.add_coefficients(coefficient_name="beta_inter",  feature_name="intercept", items_indexes=[0, 1, 3])
model.add_coefficients(coefficient_name="beta_income", feature_name="income",    items_indexes=[0, 1, 3])

# Fit the model — lbfgs is the default optimizer for ConditionalLogit
history = model.fit(canada_dataset)
report  = model.compute_report(canada_dataset)
print(report)

Question 1e

1. Sign of β_cost:
   We expect β_cost < 0. Higher cost reduces utility, so travelers should be
   less likely to choose an expensive mode. A negative coefficient is consistent
   with basic demand theory — if it came out positive, that would suggest a
   model misspecification or severe multicollinearity.

2. Intercepts across modes:
   The intercept β^inter_j captures the baseline desirability of mode j
   relative to car (the reference), after controlling for cost, time, and frequency.
   The mode with the highest intercept is the one people "default to" when all
   observable attributes are equal. In ModeCanada, car typically serves as the
   reference (β^inter_car = 0 by construction), and the other modes often show
   negative intercepts — meaning car is generally preferred as a baseline.

3. Income coefficients:
   β^income_j tells us how income shifts relative preference toward mode j vs. car.
   - Positive β^income_air → higher income people are more likely to fly (air is a luxury good)
   - Negative β^income_bus → higher income people avoid buses (bus is an inferior good)
   - β^income_train depends on the route; train sometimes competes with air for wealthier travelers
   This reveals a clear income gradient in transportation mode choice.

In [ ]:
# Question 1f

import numpy as np

car_mean_cost = transport_df.loc[transport_df["alt"] == "car", "cost"].mean()
print(f"Mean car cost = {car_mean_cost:.2f}")


probs = np.asarray(model.predict_probas(canada_dataset))
print("probs shape:", probs.shape)

if hasattr(canada_dataset, "items_ids"):
    items = list(canada_dataset.items_ids)
elif hasattr(canada_dataset, "items"):
    items = list(canada_dataset.items)
else:
    
    items = sorted(transport_df["alt"].unique().tolist())

car_index = items.index("car")
print("Items order:", items)
print("car_index =", car_index)

car_mean_prob = probs[:, car_index].mean()
print(f"Mean P(car) = {car_mean_prob:.4f}")


beta_cost = None
for w in model.trainable_weights:
    if "beta_cost" in w.name:
        beta_cost = float(w.numpy().squeeze())
        break

if beta_cost is None:
    raise ValueError("Can't find beta_cost in model.trainable_weights. Print weights' names to check.")

print(f"beta_cost = {beta_cost:.6f}")


eta_car = beta_cost * car_mean_cost * (1 - car_mean_prob)
print(f"Own-price elasticity (car) = {eta_car:.4f}")


In [ ]:
# Question 2a
# I cannot use the preprocessing="rumnet" as requested since there are errors showing "all input arrays must have the same shape"
from choice_learn.datasets import load_expedia
expedia_df = load_expedia(as_frame=True)

#take first 5000 as requested
top_ids = expedia_df["srch_id"].unique()[:5000]
expedia_df = expedia_df[expedia_df["srch_id"].isin(top_ids)].copy()

expedia_df = expedia_df.fillna(0)

print(f"Rows: {len(expedia_df)}")
print(f"Unique searches (choices): {expedia_df['srch_id'].nunique()}")
print(f"Columns: {expedia_df.shape[1]}")


sizes = expedia_df.groupby("srch_id").size()
print(f"\nChoice set size — mean: {sizes.mean():.1f}, min: {sizes.min()}, max: {sizes.max()}")


In [ ]:
# Question 2b

import tensorflow as tf
import numpy as np
import pandas as pd         
item_features = ["prop_starrating", "prop_review_score", "prop_brand_bool",
                 "prop_location_score1", "promotion_flag"]

# log(price)
expedia_df["log_price"] = np.log1p(expedia_df["price_usd"])
item_features = ["log_price"] + item_features

# ChoiceDataset
# 
MAX_ITEMS = sizes.max()  # padding

all_items_features = []
all_choices = []
all_available_items = []

for srch_id, group in expedia_df.groupby("srch_id"):
    n = len(group)
    chosen_idx = group["booking_bool"].values.argmax()

    # features shape
    feat_matrix = np.zeros((MAX_ITEMS, len(item_features)))
    feat_matrix[:n] = group[item_features].values

    # availability mask
    avail = np.zeros(MAX_ITEMS)
    avail[:n] = 1

    all_items_features.append(feat_matrix)
    all_choices.append(chosen_idx)
    all_available_items.append(avail)

all_items_features = np.array(all_items_features)   # (n_searches, MAX_ITEMS, n_features)
all_choices        = np.array(all_choices)           # (n_searches,)
all_available_items = np.array(all_available_items) # (n_searches, MAX_ITEMS)

print(f"\nFeature matrix shape: {all_items_features.shape}")
print(f"Choices shape: {all_choices.shape}")

# 80/20 split
n_total    = len(all_choices)
train_size = int(0.8 * n_total)

X_train = all_items_features[:train_size]
X_test  = all_items_features[train_size:]
y_train = all_choices[:train_size]
y_test  = all_choices[train_size:]
avail_train = all_available_items[:train_size]
avail_test  = all_available_items[train_size:]

# Conditional Logit = linear utility + softmax

class ConditionalLogitModel(tf.keras.Model):
    def __init__(self, n_features):
        super().__init__()
        self.beta = tf.keras.layers.Dense(1, use_bias=False)

    def call(self, inputs):
        return tf.squeeze(self.beta(inputs), axis=-1)

def masked_crossentropy(y_true, logits, availability):
    mask = (1 - availability) * (-1e9)
    logits_masked = logits + mask
    # cross entropy loss
    loss = tf.keras.losses.sparse_categorical_crossentropy(
        y_true, logits_masked, from_logits=True)
    return tf.reduce_mean(loss)

# standardized features
mean = X_train.mean(axis=(0,1), keepdims=True)
std  = X_train.std(axis=(0,1), keepdims=True) + 1e-8
X_train_scaled = (X_train - mean) / std
X_test_scaled  = (X_test  - mean) / std

# train
cl_model   = ConditionalLogitModel(n_features=len(item_features))
optimizer  = tf.keras.optimizers.Adam(0.01)

for epoch in range(100):
    with tf.GradientTape() as tape:
        logits = cl_model(X_train_scaled)
        loss   = masked_crossentropy(y_train, logits, avail_train)
    grads = tape.gradient(loss, cl_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, cl_model.trainable_variables))
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/100 — loss: {loss:.4f}")

# test loss
test_logits = cl_model(X_test_scaled)
cl_loss = masked_crossentropy(y_test, test_logits, avail_test).numpy()
print(f"\nConditional Logit — test cross-entropy: {cl_loss:.4f}")


In [ ]:
# Question 2c
weights = cl_model.beta.get_weights()[0].flatten()
coef_df = pd.DataFrame({"feature": item_features, "coefficient": weights})
coef_df = coef_df.sort_values("coefficient", ascending=False)
print("\nCoefficients:")
print(coef_df.to_string(index=False))

Question 2c
Symbol expectation is intuitive. The most important characteristics are usually price and review_score, as these two are the most intuitive and easily comparable signals for users to make decisions.

In [ ]:
# Question 2d

class RUMnetSimple(tf.keras.Model):
    def __init__(self, n_features):
        super().__init__()
        # 两层MLP学习非线性效用函数
        self.net = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu"),
            tf.keras.layers.Dense(1)   # 输出是scalar utility
        ])

    def call(self, inputs):
        return tf.squeeze(self.net(inputs), axis=-1)

rumnet_model = RUMnetSimple(n_features=len(item_features))
optimizer2   = tf.keras.optimizers.Adam(0.01)

for epoch in range(100):
    with tf.GradientTape() as tape:
        logits = rumnet_model(X_train_scaled)
        loss   = masked_crossentropy(y_train, logits, avail_train)
    grads = tape.gradient(loss, rumnet_model.trainable_variables)
    optimizer2.apply_gradients(zip(grads, rumnet_model.trainable_variables))
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/100 — loss: {loss:.4f}")

test_logits2 = rumnet_model(X_test_scaled)
rum_loss = masked_crossentropy(y_test, test_logits2, avail_test).numpy()

print(f"\nRUMnet (MLP)      — test cross-entropy: {rum_loss:.4f}")
print(f"Conditional Logit — test cross-entropy: {cl_loss:.4f}")
print(f"Improvement: {cl_loss - rum_loss:.4f}")


Question 2e           
RUMnet can capture nonlinearity/interaction with a large number of product features + customer features, which may theoretically be better on CE, but RUMnet needs better regularization/early stopping/hyperparameters. However, if the sample is small ( only use 5000 choices), the logit may be more stable. And for explanality, the logit is better to interpret.